In [1]:
pwd

'/home/ec2-user/SageMaker/yujin'

In [2]:
!pip install pillow torchvision boto3 pandas


In [3]:
import pandas as pd
import boto3

In [4]:
# SageMaker 노트북에 저장된 CSV 파일 경로
csv_path = '/home/ec2-user/SageMaker/NG_class.CSV'
ng_data = pd.read_csv(csv_path)
ng_data.columns = ['image_name', 'class_index']  # CSV 파일에서 열 이름 설정
print("CSV 데이터 확인:\n", ng_data.head())

CSV 데이터 확인:
    image_name  class_index
0        1058            1
1        1083            1
2        1122            1
3         115            3
4        1195            1


In [5]:
def list_all_objects(bucket_name, prefix):
    """S3에서 주어진 폴더 내 모든 객체 목록 가져오기"""
    s3 = boto3.client('s3')
    all_objects = []
    result = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
    
    if 'Contents' not in result:
        return all_objects

    all_objects.extend(result['Contents'])
    
    while result.get('IsTruncated'):
        continuation_token = result['NextContinuationToken']
        result = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix, ContinuationToken=continuation_token)
        all_objects.extend(result['Contents'])
    
    return all_objects

In [6]:
from PIL import Image
from torchvision import transforms
import os
import io

# AWS S3 클라이언트 설정
s3 = boto3.client('s3')
bucket_name = 'smwu-cv-data'
ng_folder_path = 'data/두부/NG/'

augmentation = transforms.Compose([
    transforms.RandomRotation(20),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.5, contrast=0.5),
    transforms.RandomResizedCrop((224, 224)),
    transforms.GaussianBlur(3)
])

# 전처리를 위한 변환 정의
preprocessing_transforms = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((224, 224)),
    transforms.ColorJitter(brightness=0.5, contrast=0.5),
    transforms.RandomRotation(15),
    transforms.RandomHorizontalFlip(),
    transforms.GaussianBlur(3)
])

def download_image_from_s3(bucket_name, file_key):
    """S3에서 이미지를 다운로드하여 PIL 이미지로 반환"""
    try:
        response = s3.get_object(Bucket=bucket_name, Key=file_key)
        img_data = response['Body'].read()
        return Image.open(io.BytesIO(img_data)).convert('L')
    except Exception as e:
        print(f"Error downloading {file_key}: {e}")
        return None

    

In [7]:
def augment_and_save(img, class_name, target_count, current_count, save_path):
    """이미지 증강 및 저장"""
    os.makedirs(save_path, exist_ok=True)
    augment_count = target_count - current_count

    # 증강된 이미지를 생성하고 저장
    for i in range(augment_count):
        augmented_img = augmentation(img)
        file_name = f"{class_name}_aug_{i}.bmp"
        augmented_img.save(os.path.join(save_path, file_name))
        print(f"Saved augmented image: {file_name}")


In [8]:
# 전처리 수행
def preprocess_image(img):
    return preprocessing_transforms(img)

In [9]:
# 증강된 데이터를 로컬에서 S3로 업로드
def upload_to_s3(local_folder, bucket_name, s3_folder):
    for class_folder in os.listdir(local_folder):
        class_path = os.path.join(local_folder, class_folder)
        if os.path.isdir(class_path):
            for file_name in os.listdir(class_path):
                file_path = os.path.join(class_path, file_name)
                s3_key = f"{s3_folder}/{class_folder}/{file_name}"
                s3.upload_file(file_path, bucket_name, s3_key)
                print(f"Uploaded {file_name} to s3://{bucket_name}/{s3_key}")

In [10]:
# 데이터 증강 및 전처리 실행
def preprocess_and_upload(bucket_name, folder_path, save_path):
    objects = list_all_objects(bucket_name, folder_path)

    for obj in objects:
        file_key = obj['Key']
        if file_key.endswith('.bmp'):
            img = download_image_from_s3(bucket_name, file_key)
            if img is None:
                continue

            processed_img = preprocess_image(img)
            output_path = os.path.join(save_path, os.path.basename(file_key))
            os.makedirs(os.path.dirname(output_path), exist_ok=True)
            processed_img.save(output_path)
            print(f"Processed and saved: {output_path}")

    upload_to_s3(save_path, bucket_name, 'data/전처리된_이미지')
    print("모든 이미지 전처리 및 S3 업로드 완료.")

In [11]:
# 증강된 데이터를 저장할 로컬 디렉터리
base_save_path = '/home/ec2-user/SageMaker/yujin'
target_count = 390  # Desired number of images per class
class_map = {0: 'damage', 1: 'edge', 2: 'contaminant', 3: 'bubble'}

In [ ]:
# 증강 작업 실행
for index, row in ng_data.iterrows():
    image_name = row['image_name']
    class_index = row['class_index']
    class_name = class_map[class_index]

    # S3에서 이미지 다운로드
    file_key = f"{ng_folder_path}{image_name}.bmp"
    img = download_image_from_s3(bucket_name, file_key)
    if img is None:
        continue

    # 현재 클래스의 이미지 개수 확인
    current_count = ng_data['class_index'].value_counts().get(class_index, 0)

    # 저장 경로 설정
    save_path = os.path.join(base_save_path, class_name)
    augment_and_save(img, class_name, target_count, current_count, save_path)

print("NG 데이터 증강 작업 완료.")

Saved augmented image: edge_aug_0.bmp
Saved augmented image: edge_aug_1.bmp
Saved augmented image: edge_aug_2.bmp
Saved augmented image: edge_aug_3.bmp
Saved augmented image: edge_aug_4.bmp
Saved augmented image: edge_aug_5.bmp
Saved augmented image: edge_aug_6.bmp
Saved augmented image: edge_aug_7.bmp
Saved augmented image: edge_aug_8.bmp
Saved augmented image: edge_aug_9.bmp
Saved augmented image: edge_aug_10.bmp
Saved augmented image: edge_aug_11.bmp
Saved augmented image: edge_aug_12.bmp
Saved augmented image: edge_aug_13.bmp
Saved augmented image: edge_aug_14.bmp
Saved augmented image: edge_aug_15.bmp
Saved augmented image: edge_aug_16.bmp
Saved augmented image: edge_aug_17.bmp
Saved augmented image: edge_aug_18.bmp
Saved augmented image: edge_aug_19.bmp
Saved augmented image: edge_aug_20.bmp
Saved augmented image: edge_aug_21.bmp
Saved augmented image: edge_aug_22.bmp
Saved augmented image: edge_aug_23.bmp
Saved augmented image: edge_aug_24.bmp
Saved augmented image: edge_aug_25.

Saved augmented image: edge_aug_208.bmp
Saved augmented image: edge_aug_209.bmp
Saved augmented image: edge_aug_210.bmp
Saved augmented image: edge_aug_211.bmp
Saved augmented image: edge_aug_212.bmp
Saved augmented image: edge_aug_213.bmp
Saved augmented image: edge_aug_214.bmp
Saved augmented image: edge_aug_215.bmp
Saved augmented image: edge_aug_216.bmp
Saved augmented image: edge_aug_217.bmp
Saved augmented image: edge_aug_218.bmp
Saved augmented image: edge_aug_219.bmp
Saved augmented image: edge_aug_220.bmp
Saved augmented image: edge_aug_221.bmp
Saved augmented image: edge_aug_222.bmp
Saved augmented image: edge_aug_223.bmp
Saved augmented image: edge_aug_224.bmp
Saved augmented image: edge_aug_225.bmp
Saved augmented image: edge_aug_226.bmp
Saved augmented image: edge_aug_227.bmp
Saved augmented image: edge_aug_228.bmp
Saved augmented image: edge_aug_229.bmp
Saved augmented image: edge_aug_230.bmp
Saved augmented image: edge_aug_231.bmp
Saved augmented image: edge_aug_232.bmp


Saved augmented image: edge_aug_102.bmp
Saved augmented image: edge_aug_103.bmp
Saved augmented image: edge_aug_104.bmp
Saved augmented image: edge_aug_105.bmp
Saved augmented image: edge_aug_106.bmp
Saved augmented image: edge_aug_107.bmp
Saved augmented image: edge_aug_108.bmp
Saved augmented image: edge_aug_109.bmp
Saved augmented image: edge_aug_110.bmp
Saved augmented image: edge_aug_111.bmp
Saved augmented image: edge_aug_112.bmp
Saved augmented image: edge_aug_113.bmp
Saved augmented image: edge_aug_114.bmp
Saved augmented image: edge_aug_115.bmp
Saved augmented image: edge_aug_116.bmp
Saved augmented image: edge_aug_117.bmp
Saved augmented image: edge_aug_118.bmp
Saved augmented image: edge_aug_119.bmp
Saved augmented image: edge_aug_120.bmp
Saved augmented image: edge_aug_121.bmp
Saved augmented image: edge_aug_122.bmp
Saved augmented image: edge_aug_123.bmp
Saved augmented image: edge_aug_124.bmp
Saved augmented image: edge_aug_125.bmp
Saved augmented image: edge_aug_126.bmp


Saved augmented image: edge_aug_310.bmp
Saved augmented image: edge_aug_311.bmp
Saved augmented image: edge_aug_312.bmp
Saved augmented image: edge_aug_313.bmp
Saved augmented image: edge_aug_314.bmp
Saved augmented image: edge_aug_315.bmp
Saved augmented image: edge_aug_316.bmp
Saved augmented image: edge_aug_317.bmp
Saved augmented image: edge_aug_318.bmp
Saved augmented image: edge_aug_319.bmp
Saved augmented image: edge_aug_320.bmp
Saved augmented image: edge_aug_321.bmp
Saved augmented image: edge_aug_0.bmp
Saved augmented image: edge_aug_1.bmp
Saved augmented image: edge_aug_2.bmp
Saved augmented image: edge_aug_3.bmp
Saved augmented image: edge_aug_4.bmp
Saved augmented image: edge_aug_5.bmp
Saved augmented image: edge_aug_6.bmp
Saved augmented image: edge_aug_7.bmp
Saved augmented image: edge_aug_8.bmp
Saved augmented image: edge_aug_9.bmp
Saved augmented image: edge_aug_10.bmp
Saved augmented image: edge_aug_11.bmp
Saved augmented image: edge_aug_12.bmp
Saved augmented image: 

Saved augmented image: edge_aug_200.bmp
Saved augmented image: edge_aug_201.bmp
Saved augmented image: edge_aug_202.bmp
Saved augmented image: edge_aug_203.bmp
Saved augmented image: edge_aug_204.bmp
Saved augmented image: edge_aug_205.bmp
Saved augmented image: edge_aug_206.bmp
Saved augmented image: edge_aug_207.bmp
Saved augmented image: edge_aug_208.bmp
Saved augmented image: edge_aug_209.bmp
Saved augmented image: edge_aug_210.bmp
Saved augmented image: edge_aug_211.bmp
Saved augmented image: edge_aug_212.bmp
Saved augmented image: edge_aug_213.bmp
Saved augmented image: edge_aug_214.bmp
Saved augmented image: edge_aug_215.bmp
Saved augmented image: edge_aug_216.bmp
Saved augmented image: edge_aug_217.bmp
Saved augmented image: edge_aug_218.bmp
Saved augmented image: edge_aug_219.bmp
Saved augmented image: edge_aug_220.bmp
Saved augmented image: edge_aug_221.bmp
Saved augmented image: edge_aug_222.bmp
Saved augmented image: edge_aug_223.bmp
Saved augmented image: edge_aug_224.bmp


...

Saved augmented image: edge_aug_72.bmp
Saved augmented image: edge_aug_73.bmp
Saved augmented image: edge_aug_74.bmp
Saved augmented image: edge_aug_75.bmp
Saved augmented image: edge_aug_76.bmp
Saved augmented image: edge_aug_77.bmp
Saved augmented image: edge_aug_78.bmp
Saved augmented image: edge_aug_79.bmp
Saved augmented image: edge_aug_80.bmp
Saved augmented image: edge_aug_81.bmp
Saved augmented image: edge_aug_82.bmp
Saved augmented image: edge_aug_83.bmp
Saved augmented image: edge_aug_84.bmp
Saved augmented image: edge_aug_85.bmp
Saved augmented image: edge_aug_86.bmp
Saved augmented image: edge_aug_87.bmp
Saved augmented image: edge_aug_88.bmp
Saved augmented image: edge_aug_89.bmp
Saved augmented image: edge_aug_90.bmp
Saved augmented image: edge_aug_91.bmp
Saved augmented image: edge_aug_92.bmp
Saved augmented image: edge_aug_93.bmp
Saved augmented image: edge_aug_94.bmp
Saved augmented image: edge_aug_95.bmp
Saved augmented image: edge_aug_96.bmp
Saved augmented image: ed

Saved augmented image: edge_aug_285.bmp
Saved augmented image: edge_aug_286.bmp
Saved augmented image: edge_aug_287.bmp
Saved augmented image: edge_aug_288.bmp
Saved augmented image: edge_aug_289.bmp
Saved augmented image: edge_aug_290.bmp
Saved augmented image: edge_aug_291.bmp
Saved augmented image: edge_aug_292.bmp
Saved augmented image: edge_aug_293.bmp
Saved augmented image: edge_aug_294.bmp
Saved augmented image: edge_aug_295.bmp
Saved augmented image: edge_aug_296.bmp
Saved augmented image: edge_aug_297.bmp
Saved augmented image: edge_aug_298.bmp
Saved augmented image: edge_aug_299.bmp
Saved augmented image: edge_aug_300.bmp
Saved augmented image: edge_aug_301.bmp
Saved augmented image: edge_aug_302.bmp
Saved augmented image: edge_aug_303.bmp
Saved augmented image: edge_aug_304.bmp
Saved augmented image: edge_aug_305.bmp
Saved augmented image: edge_aug_306.bmp
Saved augmented image: edge_aug_307.bmp
Saved augmented image: edge_aug_308.bmp
Saved augmented image: edge_aug_309.bmp


Saved augmented image: damage_aug_169.bmp
Saved augmented image: damage_aug_170.bmp
Saved augmented image: damage_aug_171.bmp
Saved augmented image: damage_aug_172.bmp
Saved augmented image: damage_aug_173.bmp
Saved augmented image: damage_aug_174.bmp
Saved augmented image: damage_aug_175.bmp
Saved augmented image: damage_aug_176.bmp
Saved augmented image: damage_aug_177.bmp
Saved augmented image: damage_aug_178.bmp
Saved augmented image: damage_aug_179.bmp
Saved augmented image: damage_aug_180.bmp
Saved augmented image: damage_aug_181.bmp
Saved augmented image: damage_aug_182.bmp
Saved augmented image: damage_aug_183.bmp
Saved augmented image: damage_aug_184.bmp
Saved augmented image: damage_aug_185.bmp
Saved augmented image: damage_aug_186.bmp
Saved augmented image: damage_aug_187.bmp
Saved augmented image: damage_aug_188.bmp
Saved augmented image: damage_aug_189.bmp
Saved augmented image: damage_aug_190.bmp
Saved augmented image: damage_aug_191.bmp
Saved augmented image: damage_aug_

Saved augmented image: damage_aug_371.bmp
Saved augmented image: damage_aug_372.bmp
Saved augmented image: damage_aug_373.bmp
Saved augmented image: damage_aug_374.bmp
Saved augmented image: damage_aug_375.bmp
Saved augmented image: damage_aug_376.bmp
Saved augmented image: damage_aug_377.bmp
Saved augmented image: damage_aug_378.bmp
Saved augmented image: damage_aug_379.bmp
Saved augmented image: damage_aug_0.bmp
Saved augmented image: damage_aug_1.bmp
Saved augmented image: damage_aug_2.bmp
Saved augmented image: damage_aug_3.bmp
Saved augmented image: damage_aug_4.bmp
Saved augmented image: damage_aug_5.bmp
Saved augmented image: damage_aug_6.bmp
Saved augmented image: damage_aug_7.bmp
Saved augmented image: damage_aug_8.bmp
Saved augmented image: damage_aug_9.bmp
Saved augmented image: damage_aug_10.bmp
Saved augmented image: damage_aug_11.bmp
Saved augmented image: damage_aug_12.bmp
Saved augmented image: damage_aug_13.bmp
Saved augmented image: damage_aug_14.bmp
Saved augmented i

Saved augmented image: damage_aug_194.bmp
Saved augmented image: damage_aug_195.bmp
Saved augmented image: damage_aug_196.bmp
Saved augmented image: damage_aug_197.bmp
Saved augmented image: damage_aug_198.bmp
Saved augmented image: damage_aug_199.bmp
Saved augmented image: damage_aug_200.bmp
Saved augmented image: damage_aug_201.bmp
Saved augmented image: damage_aug_202.bmp
Saved augmented image: damage_aug_203.bmp
Saved augmented image: damage_aug_204.bmp
Saved augmented image: damage_aug_205.bmp
Saved augmented image: damage_aug_206.bmp
Saved augmented image: damage_aug_207.bmp
Saved augmented image: damage_aug_208.bmp
Saved augmented image: damage_aug_209.bmp
Saved augmented image: damage_aug_210.bmp
Saved augmented image: damage_aug_211.bmp
Saved augmented image: damage_aug_212.bmp
Saved augmented image: damage_aug_213.bmp
Saved augmented image: damage_aug_214.bmp
Saved augmented image: damage_aug_215.bmp
Saved augmented image: damage_aug_216.bmp
Saved augmented image: damage_aug_